In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");


2023-10-05 20:41:25.300756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 20:41:25.300827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 20:41:25.301609: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.14.0
len(devices):  0
available GPUs: []


2023-10-05 20:41:29.289539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [2]:
import Tools.backboneUtils as backboneUtils
import Tools.loss_functions as loss_functions
import Tools.datasetTools as DatasetTools


**Dataloaders**

In [3]:
import data_setup

# try to import the dataset
dataset="SkyData"
BATCH_SIZE = 2
SHUFFLE_BUFFER_SIZE = 100

train_dataloader,test_dataloader = data_setup.create_dataloaders(dataset=dataset, 
                                                                BATCH_SIZE=BATCH_SIZE,
                                                                SHUFFLE_BUFFER_SIZE=100
                                                                )
#

[INFO] loading train dataset
[INFO] train _dataset:  27700
[INFO] loading val dataset
[INFO] val _dataset:  7990


## **Model**

In [4]:
import model_setup
import Utils
rgb_inputs_shape = (192,192,3)
ir_inputs_shape =  (128,128,3)
output_channels_per_block = 3
regression_input_shape = (*rgb_inputs_shape[:2],output_channels_per_block*2)
regression_output_shape = 8


In [5]:
# sample_regression_input = tf.zeros((1,*regression_input_shape))
# sample_backbone_input_rgb = tf.zeros((1,*rgb_inputs_shape))
# sample_backbone_input_ir = tf.zeros((1,*ir_inputs_shape))
# sample_regression_input.shape,sample_backbone_input_rgb.shape,sample_backbone_input_ir.shape

In [6]:
featureEmbeddingBackBone = model_setup.getFeatureEmbeddingBackBone(rgb_inputs_shape=rgb_inputs_shape,
                                                        ir_inputs_shape=ir_inputs_shape,
                                                        output_channels_per_block=output_channels_per_block
                                                        )

regressionHead= model_setup.getRegressionHead(input_shape=regression_input_shape,
                                                output_size=regression_output_shape
                                                )


**Visualize and save model structures**

In [7]:
# # visualize and save models

# Utils.plot_and_save_model_structure(featureEmbeddingBackBone,
#                                             save_path="resources/",
#                                             save_as=f"featureEmbeddingBackBone")
# Utils.plot_and_save_model_structure(regressionHead,
#                                             save_path="resources/",
#                                             save_as=f"regressionHead")

## **Training**


In [8]:
import engine 

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                                decay_steps=10000,
                                                                decay_rate=0.96,
                                                                staircase=True)
NUM_EPOCHS = 1

# Setup optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)



/home/nalain/miniconda3/envs/VisIrNet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# with tf.GradientTape() as tape:
model = featureEmbeddingBackBone
model.compile(optimizer=optimizer)
dataloader=train_dataloader

for batch in dataloader.take(8):
        input_images, template_images, labels,_instances = batch

        gt_matrix=DatasetTools.get_ground_truth_homographies(labels)
        warped_inputs, _ = DatasetTools._get_warped_sampled(input_images, gt_matrix)

        with tf.GradientTape() as tape:
                rgb_fmaps,ir_fmaps = model.call((input_images,template_images),training=True)
        
        
                warped_fmaps,_ = DatasetTools._get_warped_sampled(rgb_fmaps, gt_matrix)
                
                # compute similarity losses                
                _fir_frgb = loss_functions.compute_similarity_differences_mse(template_images, warped_fmaps)#should be minimal
                _fir_Iir = loss_functions.compute_similarity_differences_mse(ir_fmaps,template_images)
                _frgb_Irgb = loss_functions.compute_similarity_differences_mse(warped_fmaps,warped_inputs)#should be minimal
                _fir_Irgb = loss_functions.compute_similarity_differences_mse(ir_fmaps,warped_inputs)#should be minimal
                _frgb_Iir = loss_functions.compute_similarity_differences_mse(warped_fmaps,template_images)#
                _Iir_Irgb = loss_functions.compute_similarity_differences_mse(template_images,warped_inputs)
                
                losses_weights = [1,.001,1,1,.0000001,.0000001]
                losses = [_fir_frgb, _fir_Iir, _frgb_Irgb, _fir_Irgb, _frgb_Iir, _Iir_Irgb]
                losses = [i*j for i,j in zip(losses,losses_weights)]
                total_loss = tf.math.reduce_sum(losses)
                
                # loss shouldn't be nan
                assert not np.isnan(total_loss.numpy()), "Loss is NaN"
                
        if(tf.math.reduce_any(tf.math.is_nan(rgb_fmaps)) or tf.math.reduce_any(tf.math.is_nan(ir_fmaps))):
                print("fmaps1 or fmaps2 is nan")
                break     
        
        all_parameters= model.trainable_variables
        grads = tape.gradient(total_loss, all_parameters)
        grads = [tf.clip_by_value(i,-0.1,0.1) for i in grads]
        optimizer.apply_gradients(zip(grads, all_parameters))
        
        # create losss dictionary
        batch_losses = {"fir_frgb": _fir_frgb.numpy(),
                        "fir_Iir": _fir_Iir.numpy(),
                        "frgb_Irgb": _frgb_Irgb.numpy(),
                        "fir_Irgb": _fir_Irgb.numpy(),
                        "frgb_Iir": _frgb_Iir.numpy(),
                        "ir_Irgb": _Iir_Irgb.numpy(),
                        "total_loss": total_loss.numpy()}
        print(batch_losses)

{'fir_frgb': 201382.19, 'fir_Iir': 136397.62, 'frgb_Irgb': 139784.28, 'fir_Irgb': 88976.28, 'frgb_Iir': 201382.19, 'ir_Irgb': 18158.422, 'total_loss': 430279.2}
{'fir_frgb': 18572.82, 'fir_Iir': 5425.215, 'frgb_Irgb': 2123.418, 'fir_Irgb': 23496.57, 'frgb_Iir': 18572.82, 'ir_Irgb': 16178.701, 'total_loss': 44198.234}
{'fir_frgb': 125560.82, 'fir_Iir': 2696.491, 'frgb_Irgb': 149734.36, 'fir_Irgb': 34165.234, 'frgb_Iir': 125560.82, 'ir_Irgb': 34848.05, 'total_loss': 309463.12}
{'fir_frgb': 241356.36, 'fir_Iir': 25593.824, 'frgb_Irgb': 297382.88, 'fir_Irgb': 15821.648, 'frgb_Iir': 241356.36, 'ir_Irgb': 19590.766, 'total_loss': 554586.44}
{'fir_frgb': 180376.06, 'fir_Iir': 25393.652, 'frgb_Irgb': 189224.56, 'fir_Irgb': 22275.96, 'frgb_Iir': 180376.06, 'ir_Irgb': 32810.92, 'total_loss': 391902.0}
{'fir_frgb': 59253.25, 'fir_Iir': 6631.8164, 'frgb_Irgb': 62203.715, 'fir_Irgb': 14552.871, 'frgb_Iir': 59253.25, 'ir_Irgb': 23135.934, 'total_loss': 136016.47}
{'fir_frgb': 19599.54, 'fir_Iir': 89

In [10]:
r,i,_,_ = next(iter(test_dataloader))

In [11]:
res=model.call((r,i),training=False)